BAseline Model


In [2]:
from sklearn.datasets import fetch_rcv1

In [3]:
rcv1 = fetch_rcv1()

In [10]:
print(rcv1.DESCR)  # description of the dataset


.. _rcv1_dataset:

RCV1 dataset
------------

Reuters Corpus Volume I (RCV1) is an archive of over 800,000 manually
categorized newswire stories made available by Reuters, Ltd. for research
purposes. The dataset is extensively described in [1]_.

**Data Set Characteristics:**

==============     =====================
Classes                              103
Samples total                     804414
Dimensionality                     47236
Features           real, between 0 and 1
==============     =====================

:func:`sklearn.datasets.fetch_rcv1` will load the following
version: RCV1-v2, vectors, full sets, topics multilabels::

    >>> from sklearn.datasets import fetch_rcv1
    >>> rcv1 = fetch_rcv1()

It returns a dictionary-like object, with the following attributes:

``data``:
The feature matrix is a scipy CSR sparse matrix, with 804414 samples and
47236 features. Non-zero values contains cosine-normalized, log TF-IDF vectors.
A nearly chronological split is proposed in [1

In [12]:
print(rcv1.data.shape)    # the number of samples and features
print(rcv1.target.shape)  # the number of samples and labels


(804414, 47236)
(804414, 103)
